<a href="https://colab.research.google.com/github/alan-medina-gomez-lic/Evidencia1_Ago2025/blob/main/Pia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import json
from datetime import datetime, date, timedelta
from typing import List, Dict, Union, Any, Set
import csv
import pandas as pd
from openpyxl import Workbook, utils
from openpyxl.styles import Font, Border, Side, Alignment
import sys

TURNOS_POSIBLES: List[str] = ['MATUTINO', 'VESPERTINO', 'NOCTURNO']
TURNOS_MAP: Dict[str, str] = {'M': 'MATUTINO', 'V': 'VESPERTINO', 'N': 'NOCTURNO'}
DB_NAME: str = "reservas_coworking.db"

FECHA_FORMATO_DISPLAY: str = 'MM-DD-AAAA'
FECHA_FORMATO_INPUT: str = '%m-%d-%Y'
FECHA_HORA_FORMATO_BD: str = "%Y-%m-%d %H:%M:%S"

class SistemaReservasSQLite:
    """Clase para gestionar el sistema de reservas de salas de coworking
    utilizando SQLite para la persistencia de datos y asegurando el cumplimiento
    de todas las reglas de negocio."""

    def __init__(self):
        self.conn = sqlite3.connect(DB_NAME)
        self.cursor = self.conn.cursor()
        self.inicializar_db()

    def inicializar_db(self) -> None:
        """Inicializa la estructura de la base de datos (tablas)."""
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Clientes (
                Cliente_ID INTEGER PRIMARY KEY,
                Nombre TEXT NOT NULL,
                Apellidos TEXT NOT NULL
            )""")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Salas (
                Sala_ID INTEGER PRIMARY KEY,
                Nombre TEXT NOT NULL,
                Cupo INTEGER NOT NULL
            )""")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Reservaciones (
                Folio INTEGER PRIMARY KEY,
                Cliente_ID INTEGER NOT NULL,
                Sala_ID INTEGER NOT NULL,
                Fecha TEXT NOT NULL,
                Turno TEXT NOT NULL,
                Evento TEXT NOT NULL,
                Estado TEXT NOT NULL DEFAULT 'ACTIVA',
                FOREIGN KEY(Cliente_ID) REFERENCES Clientes(Cliente_ID),
                FOREIGN KEY(Sala_ID) REFERENCES Salas(Sala_ID)
            )""")
        self.conn.commit()
        self.cursor.execute("SELECT MAX(Folio) FROM Reservaciones")
        max_folio = self.cursor.fetchone()[0]
        self.contador_folios = (max_folio or 0) + 1
        self.cursor.execute("SELECT COUNT(*) FROM Reservaciones")
        if self.cursor.fetchone()[0] > 0:
            print(f"Buscando estado anterior en la base de datos: {DB_NAME}...")
            print("Estado anterior de la solución recuperado con éxito.")
        else:
            print(f"Buscando estado anterior en la base de datos: {DB_NAME}...")
            print("No se encontró estado anterior. Se inicia con un estado inicial vacío.")
        print("Estructura de la base de datos lista.")

    def cerrar_conexion(self) -> None:
        """Guarda el estado actual del sistema (commit y cierra conexión)."""
        try:
            self.conn.commit()
            self.conn.close()
            print("Estado de la solución almacenado (persistencia en SQLite) exitosamente.")
        except Exception as e:
            print(f"Error al cerrar la conexión/guardar el estado: {e}")

    def normalizar_fecha_str(self, fecha_str: str) -> Union[str, None]:
        """Normaliza el formato de la fecha de entrada (mm-dd-aaaa)."""
        if not fecha_str:
            return None
        fecha_normalizada = fecha_str.strip().replace('/', '-').replace(' ', '-')
        while '--' in fecha_normalizada:
            fecha_normalizada = fecha_normalizada.replace('--', '-')
        fecha_normalizada = fecha_normalizada.strip('-')
        return fecha_normalizada

    def validar_fecha_reserva(self, fecha_str: str) -> Union[datetime.date, None]:
        """Valida que la fecha de reserva sea correcta, al menos 2 días posterior a hoy y no sea domingo."""
        fecha_normalizada = self.normalizar_fecha_str(fecha_str)
        if not fecha_normalizada:
            return None
        try:
            fecha_reserva = datetime.strptime(fecha_normalizada, FECHA_FORMATO_INPUT).date()
        except ValueError:
            print(f"Error: Formato de fecha incorrecto. Use el formato {FECHA_FORMATO_DISPLAY} (ej. 10-29-2025).")
            return None
        fecha_actual_sistema = datetime.now().date()
        fecha_minima_reserva = fecha_actual_sistema + timedelta(days=2)
        if fecha_reserva < fecha_minima_reserva:
            print(f"Error: La reserva debe hacerse al menos dos días posteriores a hoy.")
            print(f"(Hoy: {fecha_actual_sistema.strftime(FECHA_FORMATO_INPUT)}. Mínimo: {fecha_minima_reserva.strftime(FECHA_FORMATO_INPUT)}).")
            return None
        if fecha_reserva.weekday() == 6:
            print("Error: No se permiten reservaciones para domingos.")
            fecha_propuesta = fecha_reserva + timedelta(days=1)
            while True:
                resp = input(f"¿Desea reservar el lunes {fecha_propuesta.strftime(FECHA_FORMATO_INPUT)}? (S/N): ").strip().upper()
                if resp == 'S':
                    return fecha_propuesta
                elif resp == 'N':
                    return None
                else:
                    print("Respuesta inválida.")
        return fecha_reserva

    def obtener_clientes_db(self) -> List[Dict[str, Any]]:
        """Devuelve un listado de clientes ordenado por apellidos y nombre."""
        self.cursor.execute("SELECT Cliente_ID, Nombre, Apellidos FROM Clientes ORDER BY Apellidos, Nombre")
        columnas = [col[0] for col in self.cursor.description]
        return [dict(zip(columnas, row)) for row in self.cursor.fetchall()]

    def obtener_salas_db(self) -> List[Dict[str, Any]]:
        """Devuelve un listado de salas con ID, nombre y cupo."""
        self.cursor.execute("SELECT Sala_ID, Nombre, Cupo FROM Salas ORDER BY Sala_ID")
        columnas = [col[0] for col in self.cursor.description]
        return [dict(zip(columnas, row)) for row in self.cursor.fetchall()]

    def obtener_slots_ocupados(self, fecha: date) -> Set[tuple]:
        """Devuelve un conjunto de (Sala_ID, Turno) ocupados para una fecha activa."""
        self.cursor.execute(
            "SELECT Sala_ID, Turno FROM Reservaciones WHERE Fecha = ? AND Estado = 'ACTIVA'",
            (fecha.strftime('%Y-%m-%d'),)
        )
        return set(self.cursor.fetchall())

    def registrar_sala(self) -> None:
        """Menú para registrar una nueva sala con validación de datos."""
        print("\n*** REGISTRAR NUEVA SALA ***")
        nombre = input("Ingrese el nombre de la sala (No puede omitirse): ").strip().upper()
        if not nombre:
            print("Error: El nombre de la sala no puede estar vacío.")
            return
        while True:
            cupo_str = input("Ingrese el cupo de la sala (Debe ser un entero > 0): ").strip()
            try:
                cupo_int = int(cupo_str)
                if cupo_int > 0:
                    break
                print("Error: El cupo debe ser un número entero mayor a cero.")
            except ValueError:
                print("Error: El cupo debe ser un número válido.")
        try:
            self.cursor.execute(
                "INSERT INTO Salas (Nombre, Cupo) VALUES (?, ?)",
                (nombre, cupo_int))
            sala_clave = self.cursor.lastrowid
            self.conn.commit()
            print(f"SALA REGISTRADA CON ÉXITO. Clave: S{sala_clave}, Nombre: '{nombre}', Cupo: {cupo_int}.")
        except sqlite3.Error as e:
            print(f"Error de base de datos al registrar sala: {e}")

    def registrar_cliente(self) -> None:
        """Menú para registrar un nuevo cliente con Nombre y Apellidos."""
        print("\n*** REGISTRAR NUEVO CLIENTE ***")
        nombre = input("Ingrese el Nombre del cliente (No puede omitirse): ").strip()
        apellidos = input("Ingrese los Apellidos del cliente (No puede omitirse): ").strip()
        if not nombre or not apellidos:
            print("Error: El Nombre y los Apellidos del cliente no pueden estar vacíos.")
            return
        try:
            self.cursor.execute(
                "INSERT INTO Clientes (Nombre, Apellidos) VALUES (?, ?)",
                (nombre, apellidos)
            )
            cliente_clave = self.cursor.lastrowid
            self.conn.commit()
            print(f"CLIENTE REGISTRADO CON ÉXITO. Clave: C{cliente_clave}. Nombre: '{nombre} {apellidos}'.")
        except sqlite3.Error as e:
            print(f"Error de base de datos al registrar cliente: {e}")

    def seleccionar_cliente(self) -> Union[int, None]:
        """Permite al usuario seleccionar un cliente registrado por su clave."""
        clientes_db = self.obtener_clientes_db()
        if not clientes_db:
            print("No hay clientes registrados. Debe registrar un cliente primero.")
            return None
        while True:
            print("\n--- CLIENTES REGISTRADOS (Clave: Apellidos, Nombre) ---")
            for cliente in clientes_db:
                print(f"[C{cliente['Cliente_ID']}]: {cliente['Apellidos']}, {cliente['Nombre']}")
            print("-----------------------------------------------------")
            clave_ingresada = input("Ingrese la CLAVE del cliente (ej. C1) o 'C' para cancelar: ").strip().upper()
            if clave_ingresada == 'C':
                print("Operación de reserva cancelada.")
                return None
            if not clave_ingresada.startswith('C') or not clave_ingresada[1:].isdigit():
                print("Error: La clave debe iniciar con 'C' seguido de un número. Intente de nuevo.")
                continue
            cliente_id_int = int(clave_ingresada[1:])
            if any(c['Cliente_ID'] == cliente_id_int for c in clientes_db):
                return cliente_id_int
            else:
                print(f"La clave '{clave_ingresada}' no existe. Intente de nuevo.")

    def registrar_reservacion(self) -> None:
        """Flujo para registrar una nueva reservación, incluyendo todas las validaciones."""
        print("\n*** REGISTRAR NUEVA RESERVACIÓN ***")
        if not self.obtener_clientes_db():
            print("Error: No hay clientes registrados.")
            return
        if not self.obtener_salas_db():
            print("Error: No hay salas registradas.")
            return
        cliente_id = self.seleccionar_cliente()
        if cliente_id is None:
            return
        fecha_almacenar: Union[date, None] = None
        while fecha_almacenar is None:
            fecha_str: str = input(f"Ingrese la FECHA de la reserva ({FECHA_FORMATO_DISPLAY}) o deje vacío para hoy: ").strip()

            if not fecha_str:
                fecha_actual_sistema: date = datetime.now().date()

                fecha_propuesta: date = fecha_actual_sistema + timedelta(days=2)

                if fecha_propuesta.weekday() == 6:
                    print(f"La fecha mínima ({fecha_propuesta.strftime(FECHA_FORMATO_INPUT)}) es domingo. Se propondrá el lunes siguiente.")
                    fecha_propuesta = fecha_propuesta + timedelta(days=1)

                fecha_propuesta_str: str = fecha_propuesta.strftime(FECHA_FORMATO_INPUT)

                resp: str = input(f"¿Desea usar la fecha mínima válida (que es {fecha_propuesta_str})? (S/N): ").strip().upper()

                if resp == 'S':
                    fecha_almacenar = fecha_propuesta
                    break
                else:
                    continue

            fecha_almacenar = self.validar_fecha_reserva(fecha_str)

        slots_ocupados: Set[tuple] = self.obtener_slots_ocupados(fecha_almacenar)
        salas_db: List[Dict[str, Any]] = self.obtener_salas_db()
        slots_disponibles: List[tuple] = []
        print(f"\n--- DISPONIBILIDAD PARA EL DÍA {fecha_almacenar.strftime(FECHA_FORMATO_INPUT)} ---")
        print("{:<5} {:<10} {:<8} {}".format("ID", "NOMBRE", "CUPO", "TURNO(S) DISPONIBLE(S)"))
        print("-" * 50)
        for sala in salas_db:
            sala_id: int = sala['Sala_ID']
            turnos_disponibles_sala: List[str] = []
            for inicial, turno in TURNOS_MAP.items():
                slot: tuple = (sala_id, turno)
                if slot not in slots_ocupados:
                    turnos_disponibles_sala.append(f"{turno} = {inicial}")
                    slots_disponibles.append(slot)
            if turnos_disponibles_sala:
                print("{:<5} {:<10} {:<8} {}".format(
                    f"S{sala_id}", sala['Nombre'], sala['Cupo'], ', '.join(turnos_disponibles_sala))
                )
            else:
                print("{:<5} {:<10} {:<8} (COMPLETO)".format(f"S{sala_id}", sala['Nombre'], sala['Cupo']))
        if not slots_disponibles:
            print("\nLo sentimos, no hay slots disponibles para esa fecha.")
            return
        self.cursor.execute(
            "SELECT Turno FROM Reservaciones WHERE Fecha = ? AND Cliente_ID = ? AND Estado = 'ACTIVA'",
            (fecha_almacenar.strftime('%Y-%m-%d'), cliente_id)
        )
        turnos_reservados_por_el_cliente: Set[str] = {row[0] for row in self.cursor.fetchall()}
        slot_elegido: Union[tuple, None] = None
        while slot_elegido is None:
            sala_id_final: Union[int, None] = None
            while sala_id_final is None:
                entrada_sala: str = input("\nIngrese la CLAVE de la sala a reservar (ej. S1) o 'C' para cancelar: ").strip().upper()
                if entrada_sala == 'C':
                    print("Operación de reserva de sala/turno cancelada.")
                    return
                if not entrada_sala.startswith('S') or not entrada_sala[1:].isdigit():
                    print("Error: La clave debe iniciar con 'S' seguido de un número. Intente de nuevo.")
                    continue
                sala_id_int: int = int(entrada_sala[1:])
                if any(s['Sala_ID'] == sala_id_int for s in salas_db):
                    sala_id_final = sala_id_int
                else:
                    print(f"Error: La clave de sala '{entrada_sala}' no existe. Intente de nuevo.")
            turnos_validos_para_prompt: List[str] = [
                t for s_id, t in slots_disponibles
                if s_id == sala_id_final and t not in turnos_reservados_por_el_cliente
            ]
            if not turnos_validos_para_prompt:
                print(f"Error: No quedan turnos disponibles en Sala S{sala_id_final} que usted no tenga reservados para esta fecha.")
                sala_id_final = None
                continue
            turnos_validos_iniciales: List[str] = [k for k, v in TURNOS_MAP.items() if v in turnos_validos_para_prompt]
            turnos_prompt_texto: str = "/".join(turnos_validos_iniciales)
            turno_elegido: Union[str, None] = None
            while turno_elegido is None:
                turno_input: str = input(f"Ingrese el TURNO (Inicial: {turnos_prompt_texto}, o nombre completo): ").strip().upper()
                turno_mapeado: Union[str, None] = None
                if turno_input in turnos_validos_iniciales:
                    turno_mapeado = TURNOS_MAP[turno_input]
                elif turno_input in TURNOS_POSIBLES:
                    turno_mapeado = turno_input
                else:
                    print(f"Error: Turno '{turno_input}' inválido. Use una inicial ({turnos_prompt_texto}) o el nombre completo (MATUTINO, VESPERTINO, NOCTURNO).")
                    continue
                if turno_mapeado in turnos_validos_para_prompt:
                    slot_elegido = (sala_id_final, turno_mapeado)
                    turno_elegido = turno_mapeado
                    break
                else:
                    print("Error: El turno ingresado no está disponible para esta sala o ya lo tiene reservado.")
        if slot_elegido is None:
            return
        while True:
            nombre_evento: str = input("Ingrese el NOMBRE del evento (No puede estar vacío): ").strip().upper()
            if nombre_evento:
                break
            print("El nombre del evento no puede estar vacío.")
        sala_id_final, turno_final = slot_elegido
        try:
            self.cursor.execute(
                "INSERT INTO Reservaciones (Cliente_ID, Sala_ID, Fecha, Turno, Evento) VALUES (?, ?, ?, ?, ?)",
                (cliente_id, sala_id_final, fecha_almacenar.strftime('%Y-%m-%d'), turno_final, nombre_evento))
            folio_unico: int = self.cursor.lastrowid
            self.conn.commit()
            print(f"\nRESERVACIÓN REGISTRADA CON ÉXITO. FOLIO: {folio_unico}")
            self.contador_folios = folio_unico + 1
        except sqlite3.Error as e:
            print(f"Error de base de datos al registrar reservación: {e}")

    def get_reservaciones_en_rango(self, fecha_inicio: date, fecha_fin: date) -> List[Dict[str, Any]]:
        """Devuelve las reservaciones activas en un rango de fechas con nombres de cliente/sala."""
        fecha_inicio_str: str = fecha_inicio.strftime('%Y-%m-%d')
        fecha_fin_str: str = fecha_fin.strftime('%Y-%m-%d')
        self.cursor.execute(
            """
            SELECT R.Folio, R.Evento, R.Fecha, S.Nombre, C.Nombre AS Cliente_Nombre, C.Apellidos, R.Turno
            FROM Reservaciones R
            JOIN Salas S ON R.Sala_ID = S.Sala_ID
            JOIN Clientes C ON R.Cliente_ID = C.Cliente_ID
            WHERE R.Fecha BETWEEN ? AND ? AND R.Estado = 'ACTIVA'
            ORDER BY R.Fecha, S.Nombre, R.Turno
            """,
            (fecha_inicio_str, fecha_fin_str))
        columnas = [col[0] for col in self.cursor.description]
        return [dict(zip(columnas, row)) for row in self.cursor.fetchall()]

    def editar_reservacion(self) -> None:
        """Permite editar el nombre del evento de una reservación ACTIVA existente en un rango de fechas."""
        print("\n*** EDITAR NOMBRE DE EVENTO ***")
        self.cursor.execute("SELECT COUNT(*) FROM Reservaciones WHERE Estado = 'ACTIVA'")
        if self.cursor.fetchone()[0] == 0:
            print("No hay reservaciones activas registradas para editar.")
            return

        fecha_inicio: Union[date, None] = None
        fecha_fin: Union[date, None] = None
        fecha_actual: date = datetime.now().date()

        while fecha_inicio is None:
            fecha_inicio_str: str = input(f"Ingrese FECHA DE INICIO del rango ({FECHA_FORMATO_DISPLAY}) o deje vacío (preguntar por Hoy): ").strip()

            if not fecha_inicio_str:
                resp: str = input(f"¿Desea usar la fecha de hoy ({fecha_actual.strftime(FECHA_FORMATO_INPUT)}) como inicio de rango? (S/N): ").strip().upper()
                if resp == 'S':
                    fecha_inicio = fecha_actual
                    break
                elif resp == 'N':
                    continue
                else:
                    print("Respuesta inválida.")
                    continue

            fecha_inicio_normalizada: Union[str, None] = self.normalizar_fecha_str(fecha_inicio_str)
            try:
                if fecha_inicio_normalizada:
                    fecha_inicio = datetime.strptime(fecha_inicio_normalizada, FECHA_FORMATO_INPUT).date()
            except (ValueError, TypeError):
                print(f"Error: Formato de fecha de inicio incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        while fecha_fin is None:
            fecha_fin_str: str = input(f"Ingrese FECHA DE FIN del rango ({FECHA_FORMATO_DISPLAY}): ").strip()

            if not fecha_fin_str:
                print("Error: Debe ingresar una fecha de fin para el rango. Intente de nuevo.")
                continue

            fecha_fin_normalizada: Union[str, None] = self.normalizar_fecha_str(fecha_fin_str)
            try:
                if fecha_fin_normalizada:
                    fecha_fin = datetime.strptime(fecha_fin_normalizada, FECHA_FORMATO_INPUT).date()
            except (ValueError, TypeError):
                print(f"Error: Formato de fecha de fin incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        if fecha_inicio is None or fecha_fin is None:
            return

        if fecha_inicio > fecha_fin:
            print("Error: La fecha de inicio no puede ser posterior a la fecha de fin. Operación cancelada.")
            return

        reservas_rango: List[Dict[str, Any]] = self.get_reservaciones_en_rango(fecha_inicio, fecha_fin)

        if not reservas_rango:
            print(f"No se encontraron eventos ACTIVOS entre {fecha_inicio.strftime(FECHA_FORMATO_INPUT)} y {fecha_fin.strftime(FECHA_FORMATO_INPUT)}.")
            return
        while True:
            print("\n--- EVENTOS ACTIVOS ENCONTRADOS ---")
            print(f"{'FOLIO':<7} {'FECHA':<15} {'EVENTO'}")
            print("----------------------------------")
            for reserva in reservas_rango:
                fecha_dt: date = datetime.strptime(reserva['Fecha'], '%Y-%m-%d').date()
                fecha_display: str = fecha_dt.strftime(FECHA_FORMATO_INPUT)
                print(f"F{reserva['Folio']:<6} {fecha_display:<15} {reserva['Evento']}")
            print("----------------------------")
            folio_a_modificar_str: str = input("Ingrese el FOLIO (ej. F1) del evento a modificar o 'C' para cancelar: ").strip().upper()
            if folio_a_modificar_str == 'C':
                print("Edición cancelada.")
                return
            if not folio_a_modificar_str.startswith('F') or not folio_a_modificar_str[1:].isdigit():
                print("Error: El folio debe iniciar con 'F' seguido de un número. Intente de nuevo.")
                continue
            folio_a_modificar_int: int = int(folio_a_modificar_str[1:])
            reserva_a_editar: Union[Dict[str, Any], None] = next((r for r in reservas_rango if r['Folio'] == folio_a_modificar_int), None)
            if reserva_a_editar:
                while True:
                    nuevo_evento: str = input(f"Ingrese el nuevo nombre para el evento 'F{folio_a_modificar_int}': ").strip().upper()
                    if nuevo_evento:
                        break
                    print("El nombre del evento no puede omitirse.")
                self.cursor.execute(
                    "UPDATE Reservaciones SET Evento = ? WHERE Folio = ? AND Estado = 'ACTIVA'",
                    (nuevo_evento, folio_a_modificar_int)
                )
                self.conn.commit()
                print(f"Evento con FOLIO F{folio_a_modificar_int} modificado a: '{nuevo_evento}'.")
                return
            else:
                print(f"Folio '{folio_a_modificar_str}' no encontrado en el rango de fechas activas. Intente de nuevo.")

    def _exportar_a_csv(self, nombre_base: str, reporte_df: pd.DataFrame) -> None:
        """Exporta el DataFrame a un archivo CSV."""
        archivo_nombre: str = f"Reporte_Reservas_{nombre_base}.csv"
        try:
            reporte_df.to_csv(archivo_nombre, index=False, encoding='utf-8')
            print(f"Reporte exportado exitosamente a CSV: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo CSV: {e}")

    def _exportar_a_json(self, nombre_base: str, reporte_df: pd.DataFrame) -> None:
        """Exporta el DataFrame a un archivo JSON."""
        archivo_nombre: str = f"Reporte_Reservas_{nombre_base}.json"
        try:
            reporte_df.to_json(archivo_nombre, orient='records', indent=4)
            print(f"Reporte exportado exitosamente a JSON: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo JSON: {e}")

    def _exportar_a_excel(self, nombre_base: str, titulo_reporte: str, reporte_df: pd.DataFrame) -> None:
        """Exporta el DataFrame a un archivo Excel aplicando el formato requerido."""
        archivo_nombre: str = f"Reporte_Reservas_{nombre_base}.xlsx"
        wb = Workbook()
        ws = wb.active
        headers: List[str] = reporte_df.columns.tolist()
        font_bold = Font(bold=True)
        thin_border = Side(border_style="thin", color="000000")
        thick_border = Side(border_style="thick", color="000000")
        border_bottom_thick = Border(top=thin_border, left=thin_border, right=thin_border, bottom=thick_border)
        center_alignment = Alignment(horizontal='center', vertical='center')

        ws['A1'] = titulo_reporte
        ws['A1'].font = font_bold
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(headers))
        ws['A1'].alignment = center_alignment

        ws.append(headers)
        header_row_num: int = ws.max_row
        for col_idx, header_text in enumerate(headers):
            col_letter: str = utils.get_column_letter(col_idx + 1)
            header_cell = ws[f'{col_letter}{header_row_num}']
            header_cell.font = font_bold
            header_cell.border = border_bottom_thick
            header_cell.alignment = center_alignment
            ws.column_dimensions[col_letter].width = max(len(header_text), 15) + 5

        for row_data in reporte_df.values.tolist():
            ws.append(row_data)
            data_row_num: int = ws.max_row
            for cell in ws[data_row_num]:
                cell.alignment = center_alignment

        try:
            wb.save(archivo_nombre)
            print(f"Reporte exportado exitosamente a Excel: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo Excel: {e}")

    def submenu_exportacion(self, nombre_base: str, titulo_reporte: str, reporte_df: pd.DataFrame) -> None:
        """Muestra el menú de opciones de exportación del reporte."""
        while True:
            exportar: str = input("\n¿Desea exportar el reporte? (C=CSV, J=JSON, E=Excel, N=No): ").strip().upper()
            if exportar == 'C':
                self._exportar_a_csv(nombre_base, reporte_df)
                break
            elif exportar == 'J':
                self._exportar_a_json(nombre_base, reporte_df)
                break
            elif exportar == 'E':
                self._exportar_a_excel(nombre_base, titulo_reporte, reporte_df)
                break
            elif exportar == 'N':
                break
            else:
                print("Opción inválida. Intente de nuevo.")

    def consultar_reservaciones(self) -> None:
        """Consulta las reservaciones activas para un rango de fechas específico y exporta el reporte."""
        print("\n*** CONSULTAR RESERVACIONES ***")

        fecha_inicio: Union[date, None] = None
        fecha_fin: Union[date, None] = None
        fecha_actual: date = datetime.now().date()

        while fecha_inicio is None:
            fecha_inicio_str: str = input(f"Ingrese FECHA DE INICIO del rango ({FECHA_FORMATO_DISPLAY}) o deje vacío (preguntar por Hoy): ").strip()

            if not fecha_inicio_str:
                resp: str = input(f"¿Desea usar la fecha de hoy ({fecha_actual.strftime(FECHA_FORMATO_INPUT)}) como inicio de rango? (S/N): ").strip().upper()
                if resp == 'S':
                    fecha_inicio = fecha_actual
                    break
                elif resp == 'N':
                    continue
                else:
                    print("Respuesta inválida.")
                    continue

            fecha_inicio_normalizada: Union[str, None] = self.normalizar_fecha_str(fecha_inicio_str)
            try:
                if fecha_inicio_normalizada:
                    fecha_inicio = datetime.strptime(fecha_inicio_normalizada, FECHA_FORMATO_INPUT).date()
            except (ValueError, TypeError):
                print(f"Error: Formato de fecha de inicio incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        while fecha_fin is None:
            fecha_fin_str: str = input(f"Ingrese FECHA DE FIN del rango ({FECHA_FORMATO_DISPLAY}): ").strip()

            if not fecha_fin_str:
                print("Error: Debe ingresar una fecha de fin para el rango. Intente de nuevo.")
                continue

            fecha_fin_normalizada: Union[str, None] = self.normalizar_fecha_str(fecha_fin_str)
            try:
                if fecha_fin_normalizada:
                    fecha_fin = datetime.strptime(fecha_fin_normalizada, FECHA_FORMATO_INPUT).date()
            except (ValueError, TypeError):
                print(f"Error: Formato de fecha de fin incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        if fecha_inicio is None or fecha_fin is None:
            return

        if fecha_inicio > fecha_fin:
            print("Error: La fecha de inicio no puede ser posterior a la fecha de fin. Operación cancelada.")
            return

        reservas_rango: List[Dict[str, Any]] = self.get_reservaciones_en_rango(fecha_inicio, fecha_fin)

        if not reservas_rango:
            rango_display: str = f"{fecha_inicio.strftime(FECHA_FORMATO_INPUT)} al {fecha_fin.strftime(FECHA_FORMATO_INPUT)}"
            print(f"No hay reservaciones ACTIVAS en el rango {rango_display}.")
            return

        nombre_base: str = f"{fecha_inicio.strftime('%Y%m%d')}_a_{fecha_fin.strftime('%Y%m%d')}"
        rango_display: str = f"{fecha_inicio.strftime(FECHA_FORMATO_INPUT)} al {fecha_fin.strftime(FECHA_FORMATO_INPUT)}"
        titulo_reporte: str = f"REPORTE DE RESERVACIONES PARA EL RANGO {rango_display}"

        data: List[List[str]] = []
        for reserva in reservas_rango:
            fecha_reserva: str = datetime.strptime(reserva['Fecha'], '%Y-%m-%d').strftime(FECHA_FORMATO_INPUT)
            cliente_nombre_completo: str = f"{reserva['Apellidos']}, {reserva['Cliente_Nombre']}"
            data.append([fecha_reserva, reserva['Nombre'], cliente_nombre_completo, reserva['Evento'], reserva['Turno']])

        columnas_df: List[str] = ['FECHA', 'SALA', 'CLIENTE', 'EVENTO', 'TURNO']
        reporte_df: pd.DataFrame = pd.DataFrame(data, columns=columnas_df)

        ANCHO_TOTAL: int = 100
        print("\n" + "*" * ANCHO_TOTAL)
        print(f"** {titulo_reporte:^94} **")
        print("*" * ANCHO_TOTAL)
        print(f"{'FECHA':<12} {'SALA':<15} {'CLIENTE':<25} {'EVENTO':<28} {'TURNO':<15}")
        print("*" * ANCHO_TOTAL)

        for _, row in reporte_df.iterrows():
            fecha: str = str(row['FECHA'])[:11]
            sala: str = str(row['SALA'])[:14]
            cliente: str = str(row['CLIENTE'])[:24]
            evento: str = str(row['EVENTO'])[:27]
            turno: str = str(row['TURNO'])[:14]
            print(f"{fecha:<12} {sala:<15} {cliente:<25} {evento:<28} {turno:<15}")

        print("*" * 17 + " FIN DEL REPORTE " + "*" * (ANCHO_TOTAL - 17 - 17 - 3) + "\n")

        self.submenu_exportacion(nombre_base, titulo_reporte, reporte_df)

    def cancelar_reservacion(self) -> None:
        """Permite cancelar una reservación activa, validando la antelación de 2 días."""
        print("\n*** CANCELAR RESERVACIÓN ***")
        self.cursor.execute("SELECT COUNT(*) FROM Reservaciones WHERE Estado = 'ACTIVA'")
        if self.cursor.fetchone()[0] == 0:
            print("No hay reservaciones activas para cancelar.")
            return

        fecha_inicio: Union[date, None] = None
        fecha_fin: Union[date, None] = None
        fecha_actual: date = datetime.now().date()

        while fecha_inicio is None:
            fecha_inicio_str: str = input(f"Ingrese FECHA DE INICIO del rango ({FECHA_FORMATO_DISPLAY}) o deje vacío (preguntar por Hoy): ").strip()

            if not fecha_inicio_str:
                resp: str = input(f"¿Desea usar la fecha de hoy ({fecha_actual.strftime(FECHA_FORMATO_INPUT)}) como inicio de rango? (S/N): ").strip().upper()
                if resp == 'S':
                    fecha_inicio = fecha_actual
                    break
                elif resp == 'N':
                    continue
                else:
                    print("Respuesta inválida.")
                    continue

            fecha_inicio_normalizada: Union[str, None] = self.normalizar_fecha_str(fecha_inicio_str)
            try:
                if fecha_inicio_normalizada:
                    fecha_inicio = datetime.strptime(fecha_inicio_normalizada, FECHA_FORMATO_INPUT).date()
            except (ValueError, TypeError):
                print(f"Error: Formato de fecha de inicio incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        while fecha_fin is None:
            fecha_fin_str: str = input(f"Ingrese FECHA DE FIN del rango ({FECHA_FORMATO_DISPLAY}): ").strip()

            if not fecha_fin_str:
                print("Error: Debe ingresar una fecha de fin para el rango. Intente de nuevo.")
                continue

            fecha_fin_normalizada: Union[str, None] = self.normalizar_fecha_str(fecha_fin_str)
            try:
                if fecha_fin_normalizada:
                    fecha_fin = datetime.strptime(fecha_fin_normalizada, FECHA_FORMATO_INPUT).date()
            except (ValueError, TypeError):
                print(f"Error: Formato de fecha de fin incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        if fecha_inicio is None or fecha_fin is None:
            return

        if fecha_inicio > fecha_fin:
            print("Error: La fecha de inicio no puede ser posterior a la fecha de fin. Operación cancelada.")
            return

        reservas_rango: List[Dict[str, Any]] = self.get_reservaciones_en_rango(fecha_inicio, fecha_fin)

        if not reservas_rango:
            print(f"No se encontraron reservaciones ACTIVAS entre {fecha_inicio.strftime(FECHA_FORMATO_INPUT)} y {fecha_fin.strftime(FECHA_FORMATO_INPUT)}.")
            return
        while True:
            print("\n--- RESERVACIONES ACTIVAS ENCONTRADAS ---")
            print(f"{'FOLIO':<7} {'FECHA':<15} {'EVENTO'}")
            print("----------------------------------")
            for reserva in reservas_rango:
                fecha_dt: date = datetime.strptime(reserva['Fecha'], '%Y-%m-%d').date()
                fecha_display: str = fecha_dt.strftime(FECHA_FORMATO_INPUT)
                print(f"F{reserva['Folio']:<6} {fecha_display:<15} {reserva['Evento']}")
            print("----------------------------")
            folio_a_cancelar_str: str = input("Ingrese el FOLIO (ej. F1) de la reserva a cancelar o 'C' para cancelar la operación: ").strip().upper()
            if folio_a_cancelar_str == 'C':
                print("Operación de cancelación abortada.")
                return
            if not folio_a_cancelar_str.startswith('F') or not folio_a_cancelar_str[1:].isdigit():
                print("Error: El folio debe iniciar con 'F' seguido de un número. Intente de nuevo.")
                continue
            folio_a_cancelar_int: int = int(folio_a_cancelar_str[1:])
            reserva_info: Union[tuple, None] = next(((r['Folio'], r['Fecha']) for r in reservas_rango if r['Folio'] == folio_a_cancelar_int), None)
            if reserva_info:
                folio: int
                fecha_reserva_db: str
                folio, fecha_reserva_db = reserva_info
                fecha_reserva_dt: date = datetime.strptime(fecha_reserva_db, '%Y-%m-%d').date()
                fecha_actual_sistema: date = datetime.now().date()
                fecha_minima_cancelacion: date = fecha_actual_sistema + timedelta(days=2)
                if fecha_reserva_dt < fecha_minima_cancelacion:
                    print(f"\n¡RECHAZADA! La cancelación debe hacerse con al menos 2 días de antelación.")
                    print(f"(Fecha de reserva: {fecha_reserva_dt.strftime(FECHA_FORMATO_INPUT)}. Se requiere cancelar antes del {fecha_minima_cancelacion.strftime(FECHA_FORMATO_INPUT)}).")
                    return
                confirmacion: str = input(f"¿CONFIRMA la cancelación de la reserva F{folio}? (S/N): ").strip().upper()
                if confirmacion == 'S':
                    try:
                        self.cursor.execute(
                            "UPDATE Reservaciones SET Estado = 'CANCELADA' WHERE Folio = ?",
                            (folio,)
                        )
                        self.conn.commit()
                        print(f"\nReservación F{folio} CANCELADA con éxito. Disponibilidad recuperada.")
                        return
                    except sqlite3.Error as e:
                        print(f"Error de base de datos al cancelar: {e}")
                        return
                elif confirmacion == 'N':
                    print("Cancelación no confirmada. Regresando al menú de selección.")
                else:
                    print("Respuesta inválida. Intente de nuevo.")
            else:
                print(f"Folio 'F{folio_a_cancelar_int}' no encontrado en el rango de fechas activas. Intente de nuevo.")

def menu_principal() -> None:
    """Función que gestiona el menú principal de la aplicación."""
    sistema = SistemaReservasSQLite()
    while True:
        print("\n" + "=" * 40)
        print("     SISTEMA DE RESERVAS COWORKING")
        print("=" * 40)
        print("[1] Registrar la reservación de una sala")
        print("[2] Editar el nombre del evento")
        print("[3] Consultar las reservaciones existentes y Exportar")
        print("[4] Cancelar una reservación")
        print("[5] Registrar a un nuevo cliente")
        print("[6] Registrar una sala")
        print("[7] Salir (Guardar Estado)")
        print("-" * 40)
        opcion: str = input("Seleccione una opción: ").strip()
        if opcion == '1':
            sistema.registrar_reservacion()
        elif opcion == '2':
            sistema.editar_reservacion()
        elif opcion == '3':
            sistema.consultar_reservaciones()
        elif opcion == '4':
            sistema.cancelar_reservacion()
        elif opcion == '5':
            sistema.registrar_cliente()
        elif opcion == '6':
            sistema.registrar_sala()
        elif opcion == '7':
            confirmacion: str = input("¿Está seguro que desea salir del sistema? (S/N): ").strip().upper()
            if confirmacion == 'S':
                sistema.cerrar_conexion()
                sys.exit(0)
            else:
                print("Operación de salida cancelada. Regresando al menú principal.")
        else:
            print("Opción inválida. Intente de nuevo.")

if __name__ == '__main__':
    try:
        menu_principal()
    except SystemExit:
        pass